# Iris flower classification with scikit-learn (run model explainer locally)

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/explain-model/explain-tabular-data-local/explain-local-sklearn-multiclass-classification.png)

Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

Explain a model with the AML explain-model package

1. Train a SVM classification model using Scikit-learn
2. Run 'explain_model' with full data in local mode, which doesn't contact any Azure services
3. Run 'explain_model' with summarized data in local mode, which doesn't contact any Azure services
4. Visualize the global and local explanations with the visualization dashboard.

In [ ]:
from sklearn.datasets import load_iris
from sklearn import svm
from azureml.explain.model.tabular_explainer import TabularExplainer

# 1. Run model explainer locally with full data

## Load the breast cancer diagnosis data

In [ ]:
iris = load_iris()
X = iris['data']
y = iris['target']
classes = iris['target_names']
feature_names = iris['feature_names']

In [ ]:
# Split data into train and test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## Train a SVM classification model, which you want to explain

In [ ]:
clf = svm.SVC(gamma=0.001, C=100., probability=True)
model = clf.fit(x_train, y_train)

## Explain predictions on your local machine

In [ ]:
tabular_explainer = TabularExplainer(model, x_train, features = feature_names, classes=classes)

## Explain overall model predictions (global explanation)

In [ ]:
global_explanation = tabular_explainer.explain_global(x_test)

In [ ]:
# Sorted SHAP values
print('ranked global importance values: {}'.format(global_explanation.get_ranked_global_values()))
# Corresponding feature names
print('ranked global importance names: {}'.format(global_explanation.get_ranked_global_names()))
# feature ranks (based on original order of features)
print('global importance rank: {}'.format(global_explanation.global_importance_rank))
# per class feature names
print('ranked per class feature names: {}'.format(global_explanation.get_ranked_per_class_names()))
# per class feature importance values
print('ranked per class feature values: {}'.format(global_explanation.get_ranked_per_class_values()))

In [ ]:
dict(zip(global_explanation.get_ranked_global_names(), global_explanation.get_ranked_global_values()))

## Explain overall model predictions as a collection of local (instance-level) explanations

In [ ]:
# feature shap values for all features and all data points in the training data
print('local importance values: {}'.format(global_explanation.local_importance_values))

## Explain local data points (individual instances)

In [ ]:
# explain the first member of the test set
instance_num = 0
local_explanation = tabular_explainer.explain_local(x_test[instance_num,:])

In [ ]:
# get the prediction for the first member of the test set and explain why model made that prediction
prediction_value = clf.predict(x_test)[instance_num]

sorted_local_importance_values = local_explanation.get_ranked_local_values()[prediction_value]
sorted_local_importance_names = local_explanation.get_ranked_local_names()[prediction_value]


dict(zip(sorted_local_importance_names, sorted_local_importance_values))

## Load visualization dashboard

In [ ]:
# Note you will need to have extensions enabled prior to jupyter kernel starting
!jupyter nbextension install --py --sys-prefix azureml.contrib.explain.model.visualize
!jupyter nbextension enable --py --sys-prefix azureml.contrib.explain.model.visualize
# Or, in Jupyter Labs, uncomment below
# jupyter labextension install @jupyter-widgets/jupyterlab-manager
# jupyter labextension install microsoft-mli-widget

In [ ]:
from azureml.contrib.explain.model.visualize import ExplanationDashboard

In [ ]:
ExplanationDashboard(global_explanation, model, x_test)